In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import zipfile

### Extracting from the Zip file

In [ ]:
%%time
with zipfile.ZipFile("/kaggle/input/dogs-vs-cats/train.zip") as train:
    train.extractall(".")
with zipfile.ZipFile("/kaggle/input/dogs-vs-cats/test1.zip") as test:
    test.extractall(".")    

In [ ]:
ls

In [ ]:
import tensorflow as tf

In [ ]:
tf.config.experimental.list_physical_devices("GPU")

In [ ]:
filenames = os.listdir('/kaggle/working/train')

In [ ]:
filenames[:5]

In [ ]:
label = []
for file in filenames:
    label.append(file.split(".")[0])

In [ ]:
label[:5]

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
encoder = LabelEncoder()
encoder.fit(label)

In [ ]:
y = encoder.transform(label)
y = y.astype(str)
# For train data generate y label must be string, list,tuple not an int

In [ ]:
y[:4]

In [ ]:
encoder.classes_

### Display the Data

In [ ]:
import random

In [ ]:
image = random.choice(filenames)
image

In [ ]:
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import load_img

In [ ]:
f  = load_img("/kaggle/working/train/"+image)
plt.imshow(f)

In [ ]:
file = {'filename':filenames,'label':y}
df = pd.DataFrame(file)

In [ ]:
df.head(2)

### Genetaring train and Validation sets

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train,y_train = train_test_split(df,test_size = 0.3,random_state = 21)

In [ ]:
y_train.head(2)

In [ ]:
y_train.info()

In [ ]:
X_train = X_train.reset_index(drop= True)
y_train = y_train.reset_index(drop= True)

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
train_data = ImageDataGenerator(rotation_range=15,
    rescale=1./255,
    shear_range=0.1,
    zoom_range=0.2,
    horizontal_flip=True,
    width_shift_range=0.1,
    height_shift_range=0.1
)

In [ ]:
train_gen = train_data.flow_from_dataframe(X_train,'/kaggle/working/train/',
                                           x_col='filename',y_col='label',target_size=(128,128),
                                          class_mode='categorical',batch_size=10)

In [ ]:
#valid_data = ImageDataGenerator(rescale=1./255)

In [ ]:
valid_gen = train_data.flow_from_dataframe(y_train,'/kaggle/working/train/',
                                           x_col='filename',y_col='label',target_size=(128,128),
                                          class_mode='categorical',batch_size=10)

### Define the MOdel-----CNN

In [ ]:
from tensorflow.keras import Sequential,layers,optimizers

In [ ]:
model = Sequential()
model.add(layers.Conv2D(filters= 64,kernel_size = 3,activation='relu',input_shape= (128,128,3),padding = 'same'))
model.add(layers.MaxPool2D(pool_size = (2,2)))
model.add(layers.Conv2D(filters= 32,kernel_size = 3,activation='relu',padding = 'same',strides = (2,2)))
model.add(layers.MaxPool2D(pool_size = (2,2)))
model.add(layers.Flatten())
model.add(layers.Dense(64,activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(32,activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(2,activation= 'sigmoid'))       

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer=optimizers.Adam(learning_rate=0.001),loss = 'categorical_crossentropy',metrics = ['accuracy'])

In [ ]:
model.fit(train_gen,validation_data=valid_gen,epochs=10,batch_size = 10,verbose=2)

In [ ]:
test_filenames = os.listdir("/kaggle/working/test1")

In [ ]:
test_df = pd.DataFrame({
    'filename': test_filenames,
})

In [ ]:
test_data = ImageDataGenerator(rescale=1./255)

In [ ]:
test_data_gen = test_data.flow_from_dataframe(
    test_df, 
    "/kaggle/working/test1", 
    x_col='filename',
    y_col=None,
    class_mode=None,
    target_size=(128,128),
    batch_size=15,
    shuffle=False
)

In [ ]:
predictions = model.predict(test_data_gen)

In [ ]:
predictions[:5]

In [ ]:
results = np.argmax(predictions,axis = 1)

In [ ]:
results[:10]

In [ ]:
from collections import Counter

In [ ]:
Counter(results)

In [ ]:
submission = pd.read_csv('/kaggle/input/dogs-vs-cats/sampleSubmission.csv')

In [ ]:
submission.head(2)

In [ ]:
label = []
for i in results:
    if i ==0:
        label.append('cat')
    else:
        label.append('dog')

In [ ]:
results.shape

In [ ]:
submission['label']= label

In [ ]:
submission.head(2)

In [ ]:
submission.to_csv("submission.csv",index = False)